In [2]:
from copy import deepcopy

In [27]:
Roraima = {
    'Uiramuta': ['Pacaraima', 'Normandia'],
    'Pacaraima': ['Uiramuta', 'Normandia', 'Boa Vista', 'Amajari'],
    'Normandia': ['Uiramuta', 'Bonfim', 'Boa Vista', 'Pacaraima'],
    'Boa Vista': ['Pacaraima', 'Normandia', 'Bonfim', 'Canta', 
                  'Mucajai', 'Alto Alegre', 'Amajari'],
    'Bonfim': ['Caracarai', 'Canta', 'Boa Vista', 'Normandia'],
    'Canta': ['Bonfim', 'Caracarai', 'Iracema', 'Mucajai', 'Boa Vista'],
    'Amajari': ['Pacaraima', 'Boa Vista', 'Alto Alegre'],
    'Alto Alegre': ['Amajari', 'Boa Vista', 'Mucajai', 'Iracema'],
    'Mucajai': ['Alto Alegre', 'Boa Vista', 'Canta', 'Iracema'],
    'Iracema': ['Alto Alegre', 'Mucajai', 'Canta', 'Caracarai'],
    'Caracarai': ['Iracema', 'Canta', 'Bonfim', 'Caroebe', 'Sao Joao da Baliza', 
                  'Sao Luiz do Anaua', 'Rorainopolis'],
    'Caroebe': ['Sao Joao da Baliza', 'Caracarai'],
    'Sao Joao da Baliza': ['Rorainopolis', 'Sao Luiz do Anaua', 'Caracarai', 'Caroebe'],
    'Rorainopolis': ['Caracarai', 'Sao Luiz do Anaua', 'Sao Joao da Baliza'],
    'Sao Luiz do Anaua': ['Caracarai', 'Sao Joao da Baliza', 'Rorainopolis'],
}

class Problema:
    def __init__(self, bordas, cores):
        # bordas é um dicionário onde cada chave é uma região
        # e seu valor é uma lista de suas regiões fronteiriças
        self.bordas = bordas

        # cores é uma lista com as 4 cores a serem usadas na coloração
        self.cores = cores

class No:
    def __init__(self, mapa, regiao, cor):
        # mapa é um dicionario
        # cada chave é uma região, seu valor é sua cor atual
        # ex: {'São Paulo': 'azul', 'Paraíba': 'vermelho'}
        # representa diferentes estados do sistema
        self.mapa = deepcopy(mapa)

        # ao criar um novo nó, passando uma região e uma cor,
        # o nó é criado com a coloração atualizada
        self.mapa[regiao] = cor

    def __str__(self):
        string = ''
        for regiao in self.mapa:
            if self.mapa[regiao] != '':
                string = string + f'{regiao}: {self.mapa[regiao]}\n'
        return string

    def __repr__(self):
        return self.__str__()
    
    # Ramifica o nó, método a ser chamado na busca
    # retorna uma lista com os filhos do nó
    def filhos(self, problema, proxima_regiao):
        filhos = []
        for cor in problema.cores:
            filho = No(self.mapa, proxima_regiao, cor)
            filhos.append(filho)
        return filhos
    
    # varre o mapa, verifica se há regiões fronteiriças com a mesma cor
    # será usado para podar nós com estados inviáveis
    def is_viavel(self, problema):
        for regiao in self.mapa:

            if self.mapa[regiao] == '':
                return True
            
            for borda in problema.bordas[regiao]:
                if self.mapa[regiao] == self.mapa[borda]:
                    return False
                
        return True

    # varre o mapa, verifica se todas as regiões estão preenchidas
    def is_preenchido(self):
        for regiao in self.mapa:

            if self.mapa[regiao] == '':
                return False
        
        return True
    
    '''
    is_viavel verifica se o nó respeita a restrição do problema: 
        não ter 4 cores iguais em regiões fronteiriças

    is_preenchido verifica se o nó está completamente colorido mas
        não verifica se o nó é viável
        
    é importante usar os métodos juntos para determinar se o nó representa
        o estado objetivo
    '''

COLORACAO_INICIANDO = 1
COLORACAO_EM_ANDAMENTO = 2
COLORACAO_SUCESSO = 3
COLORACAO_SEM_SUCESSO = 0

# classe para realizar a busca
class Coloracao:
    def __init__(self, problema):
        self.problema = problema
        self.fronteira = [] 
        #  fronteira do problema de busca

        self.status = COLORACAO_INICIANDO

        # mapa é um dicionário com todas as regiões e suas cores
        # começa em branco
        # representa o estado inicial do sistema
        self.mapa = {}
        for regiao in self.problema.bordas:
            self.mapa[regiao] = ''

    def passo(self):
        if self.status == COLORACAO_INICIANDO:

            primeira_região = list(self.mapa.keys())[0]
            # pegamos a primeira região do dicionário, que está todo em branco

            # criaremos 4 nós, para as 4 possíveis cores para a primeira região
            for cor in self.problema.cores:
                novo_no = No(self.mapa, primeira_região, cor)
                self.fronteira.append(novo_no)
            
            self.status = COLORACAO_EM_ANDAMENTO

            return
        
        try:
            proximo_no = self.fronteira.pop(-1)
        except IndexError:
            self.situacao = COLORACAO_SEM_SUCESSO
            return

        proxima_regiao = next(regiao for regiao in proximo_no.mapa if proximo_no.mapa[regiao] == '')
        # se der erro, é pq o mapa está cheio

        for filho in proximo_no.filhos(self.problema, proxima_regiao):

            # verificando se o filho é viavel antes de adicioná-lo na fronteira
            if filho.is_viavel(self.problema):

                # verificando se o filho é objetivo
                if filho.is_preenchido():
                    self.status = COLORACAO_SUCESSO
                    self.mapa = filho.mapa
                    return
                
                else: 
                    self.fronteira.append(filho)
        
        return
    
    def executar(self):
        while self.status == COLORACAO_EM_ANDAMENTO or self.status == COLORACAO_INICIANDO:
            self.passo()
        print(self.status())

        return
    
    def status(self):
        if self.status == COLORACAO_INICIANDO:
            return 'Coloração iniciando'
        elif self.status == COLORACAO_EM_ANDAMENTO:
            return 'Coloração em andamento'
        elif self.status == COLORACAO_SUCESSO:
            return 'Coloração bem sucedida'
        elif self.status == COLORACAO_SEM_SUCESSO:
            return 'Coloração sem sucesso'

In [4]:
Roraima['Uiramuta']

['Pacaraima', 'Normandia']

In [5]:
prob_ex = Problema(Roraima, ['branco', 'verde', 'azul', 'cinza'])
col_ex = Coloracao(prob_ex)
no_1 = No(col_ex.mapa, 'Uiramuta', 'branco')

In [6]:
try:
    proxima_regiao = next(regiao for regiao in no_1.mapa if no_1.mapa[regiao] == '')
except StopIteration:
    proxima_regiao = "ops"

proxima_regiao

'Pacaraima'

In [7]:
col_ex.problema.cores

['branco', 'verde', 'azul', 'cinza']

In [8]:
filhos = no_1.filhos(prob_ex, 'Pacaraima')
filhos

[Uiramuta: branco
 Pacaraima: branco,
 Uiramuta: branco
 Pacaraima: verde,
 Uiramuta: branco
 Pacaraima: azul,
 Uiramuta: branco
 Pacaraima: cinza]

In [9]:
filhos[2].is_viavel(prob_ex)

True

In [11]:
# proxima_regiao = next(regiao for regiao in no_1.mapa if no_1.mapa[regiao] == 'aa')
# proxima_regiao

In [19]:
netos = []
for filho in filhos:
    novos_netos = filho.filhos(prob_ex, 'Normandia')
    for neto in novos_netos:
        netos.append(neto)

# for neto in netos:
#     print(neto)
#     print(neto.is_objetivo())
#     print('---')


In [20]:
mapa_ex  = {
    'Uiramuta': 'a',
    'Pacaraima': 'a',
    'Normandia': 'a',
    'Boa Vista': 'a', 
    'Bonfim': 'a',
    'Canta': 'a',
    'Amajari': 'a',
    'Alto Alegre': 'a',
    'Mucajai': 'a',
    'Iracema': 'a',
    'Caracarai': 'a',
    'Caroebe': 'a',
    'Sao Joao da Baliza': 'a',
    'Rorainopolis': 'a',
    'Sao Luiz do Anaua': '',
}

In [23]:
no_cheio = No(mapa_ex, 'Sao Luiz do Anaua', 'a')
no_cheio

Uiramuta: a
Pacaraima: a
Normandia: a
Boa Vista: a
Bonfim: a
Canta: a
Amajari: a
Alto Alegre: a
Mucajai: a
Iracema: a
Caracarai: a
Caroebe: a
Sao Joao da Baliza: a
Rorainopolis: a
Sao Luiz do Anaua: a

In [24]:
no_cheio.is_objetivo()

True

In [26]:
no_cheio.is_viavel(prob_ex)

False